In [42]:
import numpy as np
import tensorflow as tf
import random as rn

np.random.seed(10)

rn.seed(10)

config = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K
tf.set_random_seed(10)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

In [2]:
import pandas as pd
flat_recall_results = []
flat_precision_results=[]
flat_accuracy_results=[]
flat_balanced_accuracy=[]
flat_f1_val=[]


stack_recall_results = []
stack_precision_results=[]
stack_accuracy_results=[]
stack_balanced_accuracy=[]
stack_f1_val=[]


ancestor_recall_results = []
ancestor_precision_results=[]
ancestor_accuracy_results=[]
ancestor_balanced_accuracy=[]
ancestor_f1_val=[]


parent_recall_results = []
parent_precision_results=[]
parent_accuracy_results=[]
parent_balanced_accuracy=[]
parent_f1_val=[]

target_recall_results = []
target_precision_results=[]
target_accuracy_results=[]
target_balanced_accuracy=[]
target_f1_val=[]

In [3]:
#plastic_data = pd.read_csv('C:/kv/Thesus/PLAStiCC/plasticc_filtered_data.csv',header=0)
plastic_data = pd.read_csv('C:/kv/Thesus/PLAStiCC/plasticc_non_filtered_data.csv',header=0)

In [4]:
#full_pla_data.columns[full_pla_data.isnull().any()]
plastic_data.columns[plastic_data.isnull().any()]
plastic_data = plastic_data.fillna(value=0.0)

In [5]:
#plastic_data.info()
#plastic_data = plastic_data.fillna(value=0)


#plastic_data = full_pla_data.fillna(value=0.0)

In [6]:
X = plastic_data[plastic_data.columns.difference(['object_id','target','parent_label','ancestor_label'])]
y = plastic_data[['target','parent_label','ancestor_label']]

X=X.astype(float)
print(X.shape)
print(y.shape)
print(np.unique(y.target,return_counts=True))


(4381, 197)
(4381, 3)
(array([42, 52, 62, 67, 90], dtype=int64), array([1193,  183,  484,  208, 2313], dtype=int64))


In [43]:
#Not required now
def flat_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=50)))
    model.add(Dense(102,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def stack_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=50)))
    model.add(Dense(105,activation='softplus'))

    model.add(Dense(5, activation='softmax'))

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def ancestor_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=50)))
   
    model.add(Dense(101,activation='softplus')) #12 10 8 

    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def parent_model():
    model = Sequential()
    model.add(Dense(200, input_dim=199, activation='softplus',kernel_initializer=glorot_normal(seed=50)))

    model.add(Dense(102,activation='softplus')) #12 60 bal

    model.add(Dense(3, activation='softmax'))
    optimizer = Adagrad(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def target_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=50)))
    model.add(Dense(105,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model


In [35]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
from keras.layers import Input,Dense,Dropout
from keras.models import Model,Sequential
from keras.initializers import glorot_normal,glorot_uniform
from keras.wrappers.scikit_learn import KerasClassifier
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam,Adagrad
from keras.constraints import maxnorm

def deep_flat_model():
    model = Sequential() #200 150 100 75 25 5
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_stack_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_ancestor_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))


    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_parent_model():
    model = Sequential()
    model.add(Dense(200, input_dim=199, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))


    model.add(Dense(3, activation='softmax'))
    optimizer = Adagrad(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_target_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model



def getTrainTestData(X,y,random_seed):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=random_seed)
    X_train=X_train.reset_index(drop=True)
    X_test =X_test.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    y_test =y_test.reset_index(drop=True)
    
    return X_train,X_test,y_train,y_test

def performScaling(X_train,X_test,y_train,y_test,col_list):

    from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
    scaler = RobustScaler()
   
    X_train_scl = scaler.fit_transform(X_train) #X_train #X_train_res
    X_test_scl = scaler.transform(X_test)

    X_train_scl = pd.DataFrame(X_train_scl, columns = col_list) #X-train_scl
    X_test_scl  = pd.DataFrame(X_test_scl, columns = col_list)

    y_train=y_train.reset_index(drop=True)
    y_test =y_test.reset_index(drop=True)

    scaled_train_df = pd.concat([X_train_scl,y_train],axis=1)
    scaled_test_df = pd.concat([X_test_scl,y_test],axis=1)
    
    return X_train_scl,X_test_scl,y_train,y_test


def performSMOTE(X_train,X_test,y_train,y_test,col_list):
   

    print("Before OverSampling, counts of label '42': {}".format(sum(y_train.target==42))) #sum(hy_train==0)
    print("Before OverSampling, counts of label '52': {}".format(sum(y_train.target==52)))

    print("Before OverSampling, counts of label '62': {} \n".format(sum(y_train.target==62)))
    print("Before OverSampling, counts of label '67': {} \n".format(sum(y_train.target==67)))
    print("Before OverSampling, counts of label '90': {} \n".format(sum(y_train.target==90)))
  



    print("Applying SMOTE")

    from imblearn.over_sampling import SMOTE, ADASYN
    sm = SMOTE(random_state=2)
    X_train_res, y_train_res = sm.fit_sample(X_train, y_train.target.ravel())

    #ad = ADASYN(sampling_strategy='auto',random_state=2)
    #X_train_res, y_train_res = ad.fit_resample(X_train, y_train.sub_label.ravel())

    print("After OverSampling, counts of label '42': {}".format(sum(y_train_res==42))) #sum(hy_train==0)
    
    print("After OverSampling, counts of label '52': {}".format(sum(y_train_res==52)))

    print("After OverSampling, counts of label '62': {} \n".format(sum(y_train_res==62)))
    print("After OverSampling, counts of label '67': {} \n".format(sum(y_train_res==67)))
    print("After OverSampling, counts of label '90': {} \n".format(sum(y_train_res==90)))



    #print(" train", np.unique(y_train_res,return_counts=True))
    
    X_train_res = pd.DataFrame(X_train_res, columns=col_list)
    y_train_res = pd.DataFrame(y_train_res, columns=['target'])

    y_train_res['parent_label']=y_train_res.target
    y_train_res['ancestor_label']=y_train_res.target
    
    
    
    y_train_res.loc[y_train_res.target==67,['parent_label']]=90
    y_train_res.loc[y_train_res.target==52,['parent_label']]=90
    
    
    y_train_res.loc[y_train_res.parent_label==90,['ancestor_label']]=100
    y_train_res.loc[y_train_res.parent_label==62,['ancestor_label']]=100
    y_train_res.loc[y_train_res.parent_label==42,['ancestor_label']]=42
    
    
    ##y_train_res=y_train_res.reset_index(drop=True)
    #y_test =y_test.reset_index(drop=True)

    smote_train_df = pd.concat([X_train_res,y_train_res],axis=1)
    smote_test_df = pd.concat([X_test,y_test],axis=1)
    
    #print(" test", np.unique(y_test,return_counts=True))

    return smote_train_df,smote_test_df

def get_estimator(model,X,y,weights):
    estimator = train_model(model,X,y)
    estimator.fit(X,y,class_weight=weights)
 
    return estimator

def train_model(base,X,y):
    
    from sklearn.model_selection import StratifiedKFold,cross_val_score  
    estimator = KerasClassifier(build_fn=base, epochs=200, batch_size=100,verbose=0)
   
        
    #kfold = StratifiedKFold(n_splits=10, shuffle=True,random_state=100)#random_state=0
    #results = cross_val_score(estimator,X,y, cv=kfold,scoring='balanced_accuracy')                              
    #print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  
    print("skipped cv")
    return estimator

def evaluate_model(classifier,X,y):
   
    
    predictions,predict_prob,accuracy,bal_accuracy,precision,recall,f1,confusion_mat = get_metrics(classifier,X,y)
    
    print_metrics(accuracy,bal_accuracy,precision,recall,f1,confusion_mat)
    
    return predictions,predict_prob,accuracy,bal_accuracy,precision,recall,f1,confusion_mat

def get_metrics(classifier,X,y):
    
    from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,balanced_accuracy_score,f1_score
    predictions,predict_prob = get_predictions(classifier,X)
    accuracy = accuracy_score(y,predictions)
    balanced_accuracy= balanced_accuracy_score(y,predictions)
    precision=precision_score(y,predictions,average=None)
    recall = recall_score(y,predictions,average=None)
    confusion_mat = confusion_matrix(y,predictions)
    f1 = f1_score(y, predictions, labels=None, pos_label=1,average=None, sample_weight=None)
    
    return predictions,predict_prob,accuracy,balanced_accuracy,precision,recall,f1,confusion_mat

def get_predictions(classifier,X):
    return classifier.predict(X), classifier.predict_proba(X)


def print_metrics(accuracy,balanced_accuracy,precision,recall,f1,confusion_mat):    
    print(" Accuracy", accuracy)
    print(" Balanced accuracy",balanced_accuracy)
    print(" Recall",recall )
    print(" Precision",precision)
    print("f1",f1)
    print(confusion_mat)
    
    
def getExperimentData(train_df,test_df,label_type):
    
    X_train = train_df[train_df.columns.difference(['ancestor_label','parent_label','target'])]
    X_test = test_df[test_df.columns.difference(['ancestor_label','parent_label','target'])]
    
    if (label_type=="ancestor"):
        
        Y_train = train_df[['ancestor_label']]
        Y_test = test_df[['ancestor_label']]
    
    elif(label_type=="parent"):
       
        Y_train = train_df[['parent_label']]
        Y_test = test_df[['parent_label']]
    
    else:
        Y_train = train_df[['target']]
        Y_test = test_df[['target']]
    
    
    print(label_type + "X_train:", X_train.shape)
    print(label_type + "Y_train:", Y_train.shape)

    print(label_type + "X_test", X_test.shape)
    print(label_type + "Y_test", Y_test.shape)

    print(label_type + "Y_train:", np.unique(Y_train,return_counts=True))
    print(label_type + "Y_test:", np.unique( Y_test,return_counts=True))
    print("\n \n")
    
    return X_train,X_test,Y_train,Y_test
    
    
    
def get_probs(dataset,predict_prob,method):
    if method=="stack":
        ip = pd.DataFrame(predict_prob, columns=['prob1','prob2','prob3','prob4','prob5'])
        X = dataset.copy()
        X['prob1']=ip[['prob1']]
        X['prob2']=ip[['prob2']]
        X['prob3']=ip[['prob3']]
        X['prob4']=ip[['prob4']]
        X['prob5']=ip[['prob5']]
        
    elif method=="ancestor":
        ip = pd.DataFrame(predict_prob, columns=['prob1','prob2'])
        X = dataset.copy()
        X['prob1']=ip[['prob1']]
        X['prob2']=ip[['prob2']]
    
    elif method=="parent":
        ip = pd.DataFrame(predict_prob, columns=['prob3','prob4','prob5'])
        X = dataset.copy()
        X['prob3']=ip[['prob3']]
        X['prob4']=ip[['prob4']]
        X['prob5']=ip[['prob5']]

        
    return X

def put_in_df(metric,hierarchy):
    
    if (hierarchy=="accuracy"):
        headers = ['accuracy','bal_accuracy']
    elif (hierarchy=="ancestor"):
        headers = ['class42','class100']
    elif (hierarchy=="parent"):
        
        headers = ['class42','class62','class90']
        
    else:
        headers= ['class42','class52','class62','class67','class90']
    
    df = pd.DataFrame(metric,columns=headers)
    
    return df


def get_weights(y_train):
    
    from sklearn.utils import class_weight
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(y_train),
                                                     y_train)


    class_weight_dict = dict(enumerate(class_weights))

    return class_weight_dict


def get_class_weights(y):
    from collections import Counter
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}


In [ ]:
#del parent_X_train,parent_X_test,parent_Y_train,parent_Y_test
#del target_X_train,target_X_test,target_Y_train,target_Y_test

In [44]:
col = X.columns
X_train,X_test,y_train,y_test = getTrainTestData(X,y,50)

X_train_scl,X_test_scl,y_train_scl,y_test_scl = performScaling(X_train,X_test,y_train,y_test,col)
scaled_train_df,scaled_test_df = performSMOTE(X_train_scl,X_test_scl,y_train_scl,y_test_scl,col)



flat_X_train,flat_X_test,flat_Y_train,flat_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"flat")
ancestor_X_train,ancestor_X_test,ancestor_Y_train,ancestor_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"ancestor")
#parent_X_train,parent_X_test,parent_Y_train,parent_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"parent")
#target_X_train,target_X_test,target_Y_train,target_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"target")

#del scaled_train_df, scaled_test_df

Before OverSampling, counts of label '42': 885
Before OverSampling, counts of label '52': 139
Before OverSampling, counts of label '62': 369 

Before OverSampling, counts of label '67': 157 

Before OverSampling, counts of label '90': 1735 

Applying SMOTE
After OverSampling, counts of label '42': 1735
After OverSampling, counts of label '52': 1735
After OverSampling, counts of label '62': 1735 

After OverSampling, counts of label '67': 1735 

After OverSampling, counts of label '90': 1735 

flatX_train: (8675, 197)
flatY_train: (8675, 1)
flatX_test (1096, 197)
flatY_test (1096, 1)
flatY_train: (array([42, 52, 62, 67, 90], dtype=int64), array([1735, 1735, 1735, 1735, 1735], dtype=int64))
flatY_test: (array([42, 52, 62, 67, 90], dtype=int64), array([308,  44, 115,  51, 578], dtype=int64))

 

ancestorX_train: (8675, 197)
ancestorY_train: (8675, 1)
ancestorX_test (1096, 197)
ancestorY_test (1096, 1)
ancestorY_train: (array([ 42, 100], dtype=int64), array([1735, 6940], dtype=int64))
ance

In [45]:
#weights = get_weights(flat_Y_train.target)
weights={}
flat_estimator = get_estimator(flat_model,flat_X_train,flat_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

flat_train_predictions,flat_train_predict_prob = get_predictions(flat_estimator,flat_X_train)

flat_predictions,flat_predict_prob,flat_accuracy,flat_bal_accuracy,flat_precision,flat_recall,flat_f1,flat_confusion_mat=evaluate_model(flat_estimator,flat_X_test,flat_Y_test)

flat_recall_results.append(flat_recall)
flat_precision_results.append(flat_precision)
flat_accuracy_results.append([flat_accuracy,flat_bal_accuracy])
flat_f1_val.append(flat_f1)




skipped cv
 Accuracy 0.6751824817518248
 Balanced accuracy 0.5238929126637867
 Recall [0.58766234 0.34090909 0.55652174 0.33333333 0.80103806]
 Precision [0.69615385 0.12396694 0.50793651 0.43589744 0.84181818]
f1 [0.63732394 0.18181818 0.53112033 0.37777778 0.82092199]
[[181  36  37   8  46]
 [  9  15   3   1  16]
 [ 13  17  64   8  13]
 [  8   5   9  17  12]
 [ 49  48  13   5 463]]


In [46]:
flat_accuracy_results

[[0.7162408759124088, 0.5427001539539946],
 [0.6879562043795621, 0.5369704135351692],
 [0.7116788321167883, 0.5385504461221688],
 [0.7062043795620438, 0.5327657409104914],
 [0.6751824817518248, 0.5238929126637867]]

In [47]:
stack_X_train = get_probs(flat_X_train,flat_train_predict_prob,"stack")
stack_X_test = get_probs(flat_X_test,flat_predict_prob,"stack")


stack_Y_train= scaled_train_df[['target']]
stack_Y_test = scaled_test_df[['target']]



print(stack_X_train.shape)
print(stack_X_test.shape)
print(np.unique(stack_Y_train,return_counts=True))

print(np.unique(stack_Y_test, return_counts=True))

(8675, 202)
(1096, 202)
(array([42, 52, 62, 67, 90], dtype=int64), array([1735, 1735, 1735, 1735, 1735], dtype=int64))
(array([42, 52, 62, 67, 90], dtype=int64), array([308,  44, 115,  51, 578], dtype=int64))


In [48]:
#weights=get_weights(stack_Y_train.target)
weights={}
stack_estimator = get_estimator(stack_model,stack_X_train,stack_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

stack_train_predictions,stack_train_predict_prob = get_predictions(stack_estimator,stack_X_train)

stack_predictions,stack_predict_prob,stack_accuracy,stack_bal_accuracy,stack_precision,stack_recall,stack_f1,stack_confusion_mat=evaluate_model(stack_estimator,stack_X_test,stack_Y_test)

stack_recall_results.append(stack_recall)
stack_precision_results.append(stack_precision)
stack_accuracy_results.append([stack_accuracy,stack_bal_accuracy])
stack_f1_val.append(stack_f1)

skipped cv
 Accuracy 0.708941605839416
 Balanced accuracy 0.5312475699417176
 Recall [0.73051948 0.25       0.52173913 0.35294118 0.80103806]
 Precision [0.62154696 0.31428571 0.55045872 0.46153846 0.84029038]
f1 [0.67164179 0.27848101 0.53571429 0.4        0.82019486]
[[225   2  28   7  46]
 [ 13  11   4   1  15]
 [ 25   7  60   8  15]
 [ 10   3   8  18  12]
 [ 89  12   9   5 463]]


In [49]:
#stack_accuracy_results.pop()
#stack_precision_results.pop()
#stack_recall_results.pop()
#stack_f1_val.pop()
stack_accuracy_results

#np.unique(ancestor_Y_train,return_counts=True)

[[0.7208029197080292, 0.535573658067682],
 [0.7098540145985401, 0.5466543006537176],
 [0.7135036496350365, 0.5367059025394647],
 [0.6897810218978102, 0.5321263276228568],
 [0.708941605839416, 0.5312475699417176]]

In [52]:
stack_recall_results

[array([0.64041096, 0.25531915, 0.46875   , 0.42857143, 0.88481675]),
 array([0.71088435, 0.19148936, 0.60606061, 0.41666667, 0.80817052]),
 array([0.72258065, 0.22      , 0.52419355, 0.39583333, 0.82092199]),
 array([0.60507246, 0.14583333, 0.66942149, 0.44230769, 0.79799666]),
 array([0.73051948, 0.25      , 0.52173913, 0.35294118, 0.80103806])]

In [82]:
#ancestor_Y_train.loc[ancestor_Y_train.ancestor_label==42,['ancestor_label']]=0
#ancestor_Y_train.loc[ancestor_Y_train.ancestor_label==100,['ancestor_label']]=1

#ancestor_Y_test.loc[ancestor_Y_test.ancestor_label==42,['ancestor_label']]=0
#ancestor_Y_test.loc[ancestor_Y_test.ancestor_label==100,['ancestor_label']]=1

#weights=get_weights(ancestor_Y_train.ancestor_label)

weights={}

ancestor_estimator = get_estimator(deep_ancestor_model,ancestor_X_train,ancestor_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

ancestor_train_predictions,ancestor_train_predict_prob = get_predictions(ancestor_estimator,ancestor_X_train)

ancestor_predictions,ancestor_predict_prob,ancestor_accuracy,ancestor_bal_accuracy,ancestor_precision,ancestor_recall,ancestor_f1,ancestor_confusion_mat=evaluate_model(ancestor_estimator,ancestor_X_test,ancestor_Y_test)

ancestor_recall_results.append(ancestor_recall)
ancestor_precision_results.append(ancestor_precision)
ancestor_accuracy_results.append([ancestor_accuracy,ancestor_bal_accuracy])
ancestor_f1_val.append(ancestor_f1)

skipped cv
 Accuracy 0.8093065693430657
 Balanced accuracy 0.7303476586470301
 Recall [0.55932203 0.90137328]
 Precision [0.67622951 0.84741784]
f1 [0.6122449  0.87356322]
[[165 130]
 [ 79 722]]


In [83]:
#weights=get_weights(ancestor_Y_train.ancestor_label)
#weights
ancestor_accuracy_results

[[0.7928832116788321, 0.68632772155611],
 [0.7956204379562044, 0.6995811855670103],
 [0.7983576642335767, 0.7160753828433477],
 [0.8166058394160584, 0.7292610310957302],
 [0.8093065693430657, 0.7303476586470301]]

In [18]:
#weights = get_class_weights(ancestor_Y_train.ancestor_label)
#weights

In [84]:
parent_X_train = get_probs(ancestor_X_train,ancestor_train_predict_prob,"ancestor")
parent_X_test = get_probs(ancestor_X_test,ancestor_predict_prob,"ancestor")


parent_Y_train= scaled_train_df[['parent_label']]
parent_Y_test = scaled_test_df[['parent_label']]


print(parent_X_train.shape)
print(parent_X_test.shape)
print(np.unique(parent_Y_train,return_counts=True))

print(np.unique(parent_Y_test, return_counts=True))

(8730, 199)
(1096, 199)
(array([42, 62, 90], dtype=int64), array([1746, 1746, 5238], dtype=int64))
(array([42, 62, 90], dtype=int64), array([295, 124, 677], dtype=int64))


In [85]:
#weights=get_weights(parent_Y_train.parent_label)
weights={}
parent_estimator = get_estimator(deep_parent_model,parent_X_train,parent_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

parent_train_predictions,parent_train_predict_prob = get_predictions(parent_estimator,parent_X_train)

parent_predictions,parent_predict_prob,parent_accuracy,parent_bal_accuracy,parent_precision,parent_recall,parent_f1,parent_confusion_mat=evaluate_model(parent_estimator,parent_X_test,parent_Y_test)

parent_recall_results.append(parent_recall)
parent_precision_results.append(parent_precision)
parent_accuracy_results.append([parent_accuracy,parent_bal_accuracy])
parent_f1_val.append(parent_f1)

skipped cv
 Accuracy 0.7326642335766423
 Balanced accuracy 0.6276451874028016
 Recall [0.56949153 0.45967742 0.85376662]
 Precision [0.66932271 0.48305085 0.79504814]
f1 [0.61538462 0.47107438 0.82336182]
[[168  25 102]
 [ 20  57  47]
 [ 63  36 578]]


In [86]:
parent_accuracy_results

[[0.7171532846715328, 0.6174234978721611],
 [0.7463503649635036, 0.6466338217357475],
 [0.7326642335766423, 0.6500290661981305],
 [0.7372262773722628, 0.6511305351813738],
 [0.7326642335766423, 0.6276451874028016]]

In [87]:
target_X_train = get_probs(parent_X_train,parent_train_predict_prob,"parent")
target_X_test = get_probs(parent_X_test,parent_predict_prob,"parent")

target_Y_train= scaled_train_df[['target']]
target_Y_test = scaled_test_df[['target']]

print(target_X_train.shape)
print(target_X_test.shape)
print(np.unique(target_Y_train,return_counts=True))
print(np.unique(target_Y_test, return_counts=True))

(8730, 202)
(1096, 202)
(array([42, 52, 62, 67, 90], dtype=int64), array([1746, 1746, 1746, 1746, 1746], dtype=int64))
(array([42, 52, 62, 67, 90], dtype=int64), array([295,  42, 124,  68, 567], dtype=int64))


In [88]:
weights={}

target_estimator = get_estimator(deep_target_model,target_X_train,target_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

target_train_predictions,target_train_predict_prob = get_predictions(target_estimator,target_X_train)

target_predictions,target_predict_prob,target_accuracy,target_bal_accuracy,target_precision,target_recall,target_f1,target_confusion_mat=evaluate_model(target_estimator,target_X_test,target_Y_test)

target_recall_results.append(target_recall)
target_precision_results.append(target_precision)
target_accuracy_results.append([target_accuracy,target_bal_accuracy])
target_f1_val.append(target_f1)

skipped cv
 Accuracy 0.6897810218978102
 Balanced accuracy 0.5095446481854997
 Recall [0.55254237 0.19047619 0.47580645 0.45588235 0.87301587]
 Precision [0.68487395 0.28571429 0.48360656 0.57407407 0.75688073]
f1 [0.61163227 0.22857143 0.4796748  0.50819672 0.81081081]
[[163  10  22   8  92]
 [  4   8  10   1  19]
 [ 19   3  59   7  36]
 [  5   2  18  31  12]
 [ 47   5  13   7 495]]


In [89]:
flat_accuracy_results


[[0.6998175182481752, 0.5658700159721741],
 [0.708029197080292, 0.5431598514825764],
 [0.7153284671532847, 0.5319163046527158],
 [0.7125912408759124, 0.529763190345413],
 [0.6943430656934306, 0.5528728095737818]]

In [91]:
stack_accuracy_results

[[0.6368613138686131, 0.5567175350534115],
 [0.7016423357664233, 0.5366728881241327],
 [0.718065693430657, 0.5344877332241443],
 [0.6843065693430657, 0.5351158097053048],
 [0.7107664233576643, 0.5592622655444941]]

In [92]:
target_accuracy_results

[[0.656934306569343, 0.510078440225562],
 [0.6578467153284672, 0.5305132436971323],
 [0.6788321167883211, 0.5014843271707056],
 [0.6897810218978102, 0.5383243248574731],
 [0.6897810218978102, 0.5095446481854997]]

In [93]:
flat_precision_results

[array([0.57142857, 0.22807018, 0.68041237, 0.76315789, 0.81913043]),
 array([0.62146893, 0.375     , 0.53153153, 0.5       , 0.83303411]),
 array([0.60294118, 0.27777778, 0.55445545, 0.625     , 0.83986371]),
 array([0.63194444, 0.18181818, 0.60638298, 0.60784314, 0.79095164]),
 array([0.65034965, 0.4375    , 0.56756757, 0.29090909, 0.83662478])]

In [94]:
stack_precision_results

[array([0.62408759, 0.10798122, 0.72727273, 0.78787879, 0.84569138]),
 array([0.62130178, 0.35294118, 0.52252252, 0.48837209, 0.82105263]),
 array([0.62883436, 0.3030303 , 0.54285714, 0.51282051, 0.83473862]),
 array([0.65217391, 0.21052632, 0.33488372, 0.625     , 0.86245353]),
 array([0.58333333, 0.4516129 , 0.57272727, 0.64444444, 0.84181818])]

In [95]:
target_precision_results

[array([0.52702703, 0.2       , 0.57608696, 0.58536585, 0.81226766]),
 array([0.72815534, 0.23728814, 0.33189655, 0.46341463, 0.82616487]),
 array([0.55976676, 0.25806452, 0.47321429, 0.47619048, 0.82922535]),
 array([0.63945578, 0.18367347, 0.51219512, 0.48214286, 0.81707317]),
 array([0.68487395, 0.28571429, 0.48360656, 0.57407407, 0.75688073])]

In [96]:
flat_recall_results

[array([0.64163823, 0.30232558, 0.515625  , 0.55769231, 0.81206897]),
 array([0.6875    , 0.30769231, 0.48760331, 0.41176471, 0.82123894]),
 array([0.6996587 , 0.21276596, 0.53333333, 0.39215686, 0.82166667]),
 array([0.61904762, 0.09090909, 0.4789916 , 0.59615385, 0.8637138 ]),
 array([0.63050847, 0.33333333, 0.50806452, 0.47058824, 0.82186949])]

In [97]:
stack_recall_results

[array([0.58361775, 0.53488372, 0.4375    , 0.5       , 0.72758621]),
 array([0.65625   , 0.30769231, 0.47933884, 0.41176471, 0.82831858]),
 array([0.6996587 , 0.21276596, 0.54285714, 0.39215686, 0.825     ]),
 array([0.6122449 , 0.09090909, 0.60504202, 0.57692308, 0.79045997]),
 array([0.71186441, 0.33333333, 0.50806452, 0.42647059, 0.81657848])]

In [98]:
target_recall_results

[array([0.66552901, 0.25581395, 0.4140625 , 0.46153846, 0.75344828]),
 array([0.46875   , 0.35897436, 0.63636364, 0.37254902, 0.8159292 ]),
 array([0.6552901 , 0.17021277, 0.5047619 , 0.39215686, 0.785     ]),
 array([0.63945578, 0.20454545, 0.52941176, 0.51923077, 0.79897785]),
 array([0.55254237, 0.19047619, 0.47580645, 0.45588235, 0.87301587])]

In [99]:
flat_f1_val

[array([0.60450161, 0.26      , 0.58666667, 0.64444444, 0.81558442]),
 array([0.65281899, 0.33802817, 0.50862069, 0.4516129 , 0.82709447]),
 array([0.64770932, 0.24096386, 0.54368932, 0.48192771, 0.83066554]),
 array([0.62542955, 0.12121212, 0.53521127, 0.60194175, 0.8257329 ]),
 array([0.64027539, 0.37837838, 0.53617021, 0.35955056, 0.82918149])]

In [100]:
stack_f1_val

[array([0.6031746 , 0.1796875 , 0.54634146, 0.61176471, 0.78220575]),
 array([0.63829787, 0.32876712, 0.5       , 0.44680851, 0.8246696 ]),
 array([0.66235864, 0.25      , 0.54285714, 0.44444444, 0.82984074]),
 array([0.63157895, 0.12698413, 0.43113772, 0.6       , 0.82488889]),
 array([0.64122137, 0.38356164, 0.53846154, 0.51327434, 0.82900627])]

In [101]:
target_f1_val

[array([0.58823529, 0.2244898 , 0.48181818, 0.51612903, 0.78175313]),
 array([0.57034221, 0.28571429, 0.43626062, 0.41304348, 0.82101514]),
 array([0.60377358, 0.20512821, 0.48847926, 0.43010753, 0.80650685]),
 array([0.63945578, 0.19354839, 0.52066116, 0.5       , 0.8079242 ]),
 array([0.61163227, 0.22857143, 0.4796748 , 0.50819672, 0.81081081])]

In [90]:

ancestor_precision_df=put_in_df(ancestor_precision_results,"ancestor")
ancestor_recall_df=put_in_df(ancestor_recall_results,"ancestor")
ancestor_f1_df=put_in_df(ancestor_f1_val,"ancestor")


parent_precision_df=put_in_df(parent_precision_results,"parent")
parent_recall_df=put_in_df(parent_recall_results,"parent")
parent_f1_df=put_in_df(parent_f1_val,"parent")

target_precision_df=put_in_df(target_precision_results,"target")
target_recall_df=put_in_df(target_recall_results,"target")
target_f1_df=put_in_df(target_f1_val,"target")


flat_precision_df=put_in_df(flat_precision_results,"flat")
flat_recall_df=put_in_df(flat_recall_results,"flat")
flat_f1_df=put_in_df(flat_f1_val,"flat")


stack_precision_df=put_in_df(stack_precision_results,"flat")
stack_recall_df=put_in_df(stack_recall_results,"flat")
stack_f1_df=put_in_df(stack_f1_val,"flat")


flat_acc_df = put_in_df(flat_accuracy_results,"accuracy")
stack_acc_df = put_in_df(stack_accuracy_results,"accuracy")
ancestor_acc_df = put_in_df(ancestor_accuracy_results,"accuracy")
parent_acc_df = put_in_df(parent_accuracy_results,"accuracy")
target_acc_df = put_in_df(target_accuracy_results,"accuracy")

flat_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_flat_precision.csv')
flat_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_flat_recall1.csv')
flat_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_flat_f1_1.csv')

stack_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_stack_precision.csv')
stack_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_stack_recall.csv')
stack_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_stack_f1.csv')


ancestor_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_ancestor_precision.csv')
ancestor_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_ancestor_recall.csv')
ancestor_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_ancestor_f1.csv')


parent_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_parent_precision.csv')
parent_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_parent_recall.csv')
parent_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_parent_f1.csv')


target_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_target_precision.csv')
target_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_target_recall.csv')
target_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_target_f1.csv')

flat_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_flat_acc.csv')
stack_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_stack_acc.csv')
ancestor_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_ancestor_acc.csv')
parent_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_parent_acc.csv')
target_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/smote/6deep_target_acc.csv')



del /
stack_precision_df,stack_recall_df,stack_f1_df,/
flat_precision_df,flat_recall_df,flat_f1_df,/
ancestor_precision_df,ancestor_recall_df,ancestor_f1_df,/
parent_precision_df,parent_recall_df,parent_f1_df,/
target_precision_df,target_recall_df,target_f1_df,/
flat_acc_df,stack_acc_df,ancestor_acc_df,parent_acc_df,target_acc_df/

NameError: name 'parent_acc_df' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score,make_scorer,f1_score

model = KerasClassifier(build_fn=flat_model,verbose=0)
batch_size = [100, 500, 1000, 2000]
epochs = [100,200,300]
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#param_grid = dict(optimizer=optimizer)
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
#units = [8,10,12,14,16]
#param_grid = dict(units=units)
#weight_constraint = [1, 2, 3, 4, 5]
#dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(batch_size=batch_size,epochs=epochs)
#scoring = {'bal_accuracy': make_scorer(balanced_accuracy_score)}
#f1_scorer = make_scorer(f1_score,average='micro',greater_is_better=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(flat_X_train,flat_Y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:

#Best: 0.701065 using {'optimizer': 'Adam'}
#0.666058 (0.011317) with: {'optimizer': 'SGD'}
#0.688280 (0.012305) with: {'optimizer': 'RMSprop'}
#0.674886 (0.008202) with: {'optimizer': 'Adagrad'}
#0.692542 (0.007165) with: {'optimizer': 'Adadelta'}
#0.701065 (0.007761) with: {'optimizer': 'Adam'}
#0.686454 (0.008642) with: {'optimizer': 'Adamax'}
#0.687671 (0.008066) with: {'optimizer': 'Nadam'}


#Best: 0.690107 using {'learn_rate': 0.001}
#0.690107 (0.006766) with: {'learn_rate': 0.001}
#0.656621 (0.006501) with: {'learn_rate': 0.01}
#0.603957 (0.028003) with: {'learn_rate': 0.1}
#0.542770 (0.012705) with: {'learn_rate': 0.2}
#0.512329 (0.032133) with: {'learn_rate': 0.3}



#Best: 0.704110 using {'init_mode': 'normal'}
#0.693455 (0.010607) with: {'init_mode': 'uniform'}
#0.687976 (0.006027) with: {'init_mode': 'lecun_uniform'}
#0.704110 (0.016856) with: {'init_mode': 'normal'}
#0.529680 (0.009041) with: {'init_mode': 'zero'}
#0.697717 (0.008469) with: {'init_mode': 'glorot_normal'}
#0.695586 (0.008214) with: {'init_mode': 'glorot_uniform'}
#0.693151 (0.012455) with: {'init_mode': 'he_normal'}
#0.692542 (0.006073) with: {'init_mode': 'he_uniform'}

#Best: 0.702892 using {'activation': 'softplus'}
#0.654795 (0.009163) with: {'activation': 'softmax'}
#0.702892 (0.003444) with: {'activation': 'softplus'}
#0.698630 (0.006834) with: {'activation': 'softsign'}
#0.689193 (0.018300) with: {'activation': 'relu'}
#0.695282 (0.008642) with: {'activation': 'tanh'}
#0.691020 (0.004965) with: {'activation': 'sigmoid'}
#0.683714 (0.005496) with: {'activation': 'hard_sigmoid'}
#0.650228 (0.001292) with: {'activation': 'linear'}


#Best: 0.708371 using {'units': 12}
#0.699239 (0.009442) with: {'units': 8}
#0.698630 (0.002237) with: {'units': 10}
#0.708371 (0.007616) with: {'units': 12}
#0.706240 (0.006766) with: {'units': 14}
#0.705632 (0.005792) with: {'units': 16}